# Curso: Bioestatística — Experimento de Regressão Linear
## Autores: Sandro da Silva Camargo e Fernando Cardoso

**Problema**:Um experimento foi realizado para testar o efeito
da adubação nitrogenada (0, 100, 200 e 300 kg/ha de Adubo
Nitrogenado) na produção de milho (kg/parcela).


A base de dados está disponível [aqui](https://github.com/Sandrocamargo/biostatistics/blob/master/datasets/rl-milho.txt).

Abra este código no seu google colab [clicando aqui](https://colab.research.google.com/github/Sandrocamargo/biostatistics/blob/master/python/bioe_06_Regressao_Linear_Milho.ipynb).

In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

# Carga e inspeção dos dados

In [ ]:
# === 1. Leitura dos dados ===
dados = pd.read_csv("https://raw.githubusercontent.com/Sandrocamargo/biostatistics/refs/heads/master/datasets/rl-milho.txt", sep=r"\s+", header=0)
dados.info()
dados.head()

# Adaptação de um modelo de regressão linear

In [ ]:
dados["trat"] = dados["dose"].astype("category")

modelo = smf.ols("prod ~ dose", data=dados).fit()
print("=== Modelo de regressão simples ===")
print(modelo.summary())

**📊 Resumo do modelo**

Equação estimada:
$\hat{prod} = 49.975 + 0.0572 × dose$
	​
1. Coeficientes e significância
|Parâmetro	|Estimativa	|Erro padrão	|t|p-valor|Interpretação|
|:--|:--|:--|:--|:--|:--
|Intercepto |49.975	| 2.282	|21.90|	< 0.001	|Quando a dose é 0, a produção média esperada é cerca de 49.98 unidades.|
|Dose |	0.0572	| 0.012	|4.69|	< 0.001	|A cada aumento de 1 unidade na dose, a produção aumenta em média 0.0572 unidades, de forma estatisticamente significativa.|

**O p-valor < 0.001 indica que a variável dose tem efeito altamente significativo sobre a produção (prod).**

2. Qualidade do ajuste
|Indicador|Valor|Interpretação|
|:--|:--|:--|
|R² |= 0.612	|O modelo explica 61,2% da variação total da produção.	|
|R² ajustado |= 0.584	|Considerando o número de observações, o modelo ainda explica 58,4% da variação — um ajuste moderado a bom.	|
|F(1,14) |= 22.04; p = 0.000345|	O modelo como um todo é estatisticamente significativo, ou seja, há relação linear entre dose e produção.	|

**💡 Em termos práticos: mais da metade da variação na produção é explicada apenas pela dose de adubo.**

3. Diagnóstico dos resíduos
|Estatística|Valor|Interpretação|
|:--|:--|:--|
|Durbin-Watson |= 2.62	|Indica ausência de autocorrelação nos resíduos (valor próximo de 2 é ideal).	|
|Omnibus |= 2.27, JB = 0.58, p = 0.75	|Os resíduos seguem distribuição normal, o que valida os testes t e F.	|
|Cond. No. |= 313|	Não há sinais relevantes de multicolinearidade, pois é um modelo simples.

**🧩 Conclusão geral**
* O modelo é estatisticamente significativo (p < 0.001).
* A variável dose exerce um efeito positivo e relevante sobre a produção.
* O ajuste é razoavelmente bom (R² ≈ 0.61).
* Os resíduos atendem aos pressupostos de normalidade e independência.

**🧠 Interpretação**

* À medida que a dose de adubo aumenta, a produção também aumenta de forma significativa.

* O modelo explica cerca de 60% da variação na produção, e os testes estatísticos indicam que a relação é consistente e confiável.

# Análise de variância

In [ ]:
print("\n=== ANOVA usual ===")
anova_geral = sm.stats.anova_lm(modelo, typ=2)
print(anova_geral)

1. Teste F global

Hipóteses:

*  H₀: o coeficiente da variável dose é igual a zero (sem efeito sobre prod);

*  H₁: o coeficiente da variável dose é diferente de zero.
*  F = 22.04 e p = 0.000345 ⟹ rejeita-se H₀.

**Portanto, há evidência altamente significativa (p < 0.001) de que a dose influencia a produção.**

2. Importância da SQ

Soma dos quadrados de regressão (SQTrat ou SQDose) = 655.51
→ representa a variação explicada pela relação entre dose e prod.

Soma dos quadrados do resíduo (SQRes) = 416.43
→ é a variação não explicada (erro).

**💡 O fato de a SQDose ser maior que a SQRes indica que a maior parte da variação total da produção é explicada pela dose.**

3. R² implícito

Podemos obter o R² diretamente da ANOVA:

$𝑅^2 = \frac{SQ_{Regressao}}{SQ_{Total}} = \frac{655.51}{655.51+416.43} = 0.6119$​

* O mesmo valor encontrado no modelo de regressão: R² = 0.612.

**✅ Conclusão**
*  O modelo é estatisticamente significativo (F = 22.04; p < 0.001).
*  A variável dose explica cerca de 61% da variação na produção.

**Assim, há uma relação linear forte e positiva entre a dose de adubo e a produção.**

In [ ]:
# === 3. Decomposição da SQTratamento ===
# SQTratamento = SQRegressão + SQFalta de Ajuste
desd_modelo = smf.ols("prod ~ dose + trat", data=dados).fit()
anova_desd = sm.stats.anova_lm(desd_modelo, typ=2)
print("\n=== ANOVA com decomposição (dose + trat) ===")
print(anova_desd)

# Coeficiente de determinação (r²)
SQReg = anova_desd.loc["dose", "sum_sq"]
SQTrat = SQReg + anova_desd.loc["trat", "sum_sq"]
r2 = SQReg / SQTrat
print(f"\nCoeficiente de Determinação (r²) = {r2:.3f}")

Este resultado representa uma ANOVA com decomposição, usada quando há repetição para cada dose de adubo. Ela separa o efeito total de “tratamentos” (as doses) em duas partes:
*  variação explicada pela regressão (dose), e
*  variação de falta de ajuste (trat – dose).


🔹 1. Interpretação geral

*  O modelo testa se a produção de milho varia com a dose de adubo e se essa variação é adequadamente explicada por um modelo linear.
*  O efeito de trat (todas as doses) é significativo (p = 0.007): há diferenças entre as médias das doses.
*  O efeito de dose (parte linear) também é altamente significativo (p = 0.0016): há relação linear clara entre dose e produção.

🔹 2. Decomposição da variação

*  A soma de quadrados total dos tratamentos foi desdobrada assim:

$SQ_{Trat} = SQ_{Regressao} + SQ_{Falta de Ajuste}$

onde:

* $SQ_{Trat}$ = 666.69

* $SQ_{Regressão (dose)}$ = 551.12 → parte explicada pela relação linear

* $SQ_{Falta de ajuste}$ = 666.69 − 551.12 = 115.57

➡️ A falta de ajuste (diferença entre o modelo linear e as médias observadas) não é significativa, indicando que o modelo linear é adequado para descrever a relação entre dose e produção.

🔹 3. Coeficiente de determinação (r² = 0.453)

*  O r² mostra que aproximadamente 45,3% da variação total da produção é explicada pela regressão linear com a dose.
*  Embora significativo, isso indica que há variabilidade residual considerável, possivelmente devido a outros fatores experimentais não incluídos no modelo.

✅ Conclusão

* Há diferenças significativas entre as doses (p = 0.007).
* O efeito linear da dose é altamente significativo (p = 0.0016).
* O modelo linear explica cerca de 45% da variação da produção, sendo estatisticamente adequado (sem falta de ajuste relevante).

Assim, pode-se concluir que aumentos na dose de adubo estão associados a aumentos proporcionais na produção, dentro da faixa estudada.

# Análise gráfica

In [ ]:
# === 4. Ajuste final e gráfico ===
modelo_ajustado = modelo  # mesmo modelo linear simples

b0, b1 = modelo_ajustado.params
print(f"\nModelo ajustado: y = {b0:.5f} + {b1:.5f}x")

# Geração do gráfico
plt.figure(figsize=(8, 6))
plt.scatter(dados["dose"], dados["prod"], color="blue", label="Observações")

# linha de regressão
x_vals = pd.Series(sorted(dados["dose"]))
y_vals = b0 + b1 * x_vals
plt.plot(x_vals, y_vals, color="red", label="Ajuste linear")

plt.xlabel("Dose adubo nitrogenado (kg/ha)")
plt.ylabel("Produção de milho (kg/parcela)")
plt.title(f"R² ajustado = {modelo_ajustado.rsquared_adj:.4f}")
plt.legend(title=f"y = {b0:.5f} + {b1:.5f}x", loc="upper left")
plt.grid(True)
plt.tight_layout()
plt.show()